In [ ]:
## Importing Libraries
import pandas as pd
import numpy as np
import seaborn as snsb
import matplotlib.pyplot as plt
import os

## 1.Importing FINRA pdf files

FINRA operates the largest securities dispute resolution forum in the United States, and has extensive experience in providing a fair, efficient and effective venue to handle a securities-related dispute.

All cases related to Arbitraion and Mediation can be found online and can be scraped.

### Scraping FINRA pdf case files

In [ ]:
for i in range(1):
    tab_df, = pd.read_html('https://www.finra.org/arbitration-and-mediation/arbitration-awards-online?page="+str(i)')
    tab_df.to_csv("table.csv", index=False)


In [ ]:
## importing the cleaned data set
m_df= pd.read_csv('data/finra_data.csv')


## 2. Extracting Features and Labels

### Forming groups

In [ ]:
judges_list =[]
for line in open('data/list_judges.txt').readlines():
    judges_list.append(line.replace("\n",""))
    
## NAR list
nar_list =[]
for line in open('data/nar_list.txt').readlines():
    nar_list.append(line.replace("\n",""))

In [ ]:
def check_member_lawyer(row):
    
    member_lawyer_flag= False
    claimant_reps = row['Claimant_Reps']
    claimant_reps_list = str(claimant_reps).split(",")
    for c_i in claimant_reps_list:
        ## check if any of the claimants in the judges list
        for judge in judges_list:
            if Levenshtein.ratio(c_i,judge) >= 0.90:
                member_lawyer_flag= True
    ## 
    respondent_reps = row['Respondent_Reps']
    respondent_reps_list = str(respondent_reps).split(",")
    for r_i in respondent_reps_list:
        ## check if any of the claimants in the judges list
        for judge in judges_list:
            if Levenshtein.ratio(r_i,judge) >= 0.90:
                member_lawyer_flag= True
    return member_lawyer_flag

m_df['is_present_in_judge_list'] = m_df.apply(check_member_lawyer,axis = 1)

In [ ]:
def check_esq(row):
    esq_flag= False
    file_path = '/data/raw/sentences_finra_forum/'+ row['file_id']
    f = open(file_path,'r').read().lower()
    if ('esq' in f):
        esq_flag = True
    return esq_flag
    
    
    

m_df['is_esq'] = m_df.apply(check_esq,axis = 1)

In [ ]:
def check_NAR(row):
    
    nar_flag= False
    claimant_reps = row['Claimant_Reps']
    claimant_reps_list = str(claimant_reps).split(",")
    for c_i in claimant_reps_list:
        ## check if any of the claimants in the judges list
        for judge in nar_list:
            if Levenshtein.ratio(c_i,judge) >= 0.90:
                nar_flag= True
    ## 
    respondent_reps = row['Respondent_Reps']
    respondent_reps_list = str(respondent_reps).split(",")
    for r_i in respondent_reps_list:
        ## check if any of the claimants in the judges list
        for judge in nar_list:
            if Levenshtein.ratio(r_i,judge) >= 0.90:
                nar_flag= True
    return nar_flag

m_df['is_present_in_nar_list'] = m_df.apply(check_member_lawyer,axis = 1)

In [ ]:
m_df['new_group'] = None
for i in m_df.index:
    if ((m_df.loc[i,'is_claimant_matches_claimant_reps']==True) & (m_df.loc[i,'count_no_of_claimant_reps']==1)):
        m_df.loc[i,'new_group'] = 'Group 1: Pro Se'
    elif ((m_df.loc[i,'is_claimant_matches_claimant_reps']==False) \
          & (m_df.loc[i,'count_no_of_claimant_reps']==1)\
         & (m_df.loc[i,'is_present_in_nar_list']==True)):
        m_df.loc[i,'new_group'] = 'Group 2: One Non-Attorney'
    elif ((m_df.loc[i,'is_claimant_matches_claimant_reps']==False) \
          & (m_df.loc[i,'count_no_of_claimant_reps']==1)\
          & (m_df.loc[i,'is_present_in_nar_list']==False)\
         & (m_df.loc[i,'is_esq']==True)):
        m_df.loc[i,'new_group'] = 'Group 3: One Attorney'
    elif (m_df.loc[i,'count_no_of_claimant_reps']>1):
        m_df.loc[i,'new_group'] = 'Group 4: Multiple Representatives'
        

### Getting Regions from States

In [ ]:
m_df['state'] = m_df['Hearing_Site'].apply(lambda x:str(x).split(",")[-1].replace(" ",""))
region_df = pd.read_csv('data/raw/state_regions.csv')

m_df = pd.merge(m_df,region_df, left_on ='state',right_on='State Code',how='left')

### Binning into Claim Types

In [ ]:
claim_df= pd.read_excel('claims.xlsx')
claim_df = claim_df[['AwardID','Claim_Type']]
claim_df = claim_df.drop_duplicates(subset=['AwardID', 'Claim_Type'], keep=False)

In [ ]:
m_df = pd.merge(m_df,claim_df, on='AwardID',how='left')

In [ ]:
def clean_claims(row):
    claim_type = str(row['Claim_Type'])
    str_claims = claim_type.replace("[","").replace('"','').replace("'","").replace("and"," ").replace("  "," ").replace("]","").lower().strip()
    claim_type = str_claims.split(",")
    claim_type = [cl.strip() for cl in claim_type]
    return claim_type


m_df['Claim_Type_cleaned'] = m_df.apply(clean_claims,axis=1)

In [ ]:
claim_dict = {'Claim Type 1: Breach of Contract,Breach of Implied Contract':['breach of contract',\
'breach of the covenant of good faith fair dealing',\
'breach of contract unjust enrichment',\
'breach of implied covenant of good faith fair dealing',\
'breach of the implied covenant of good faith fair dealing',\
'breach of covenant of good faith fair dealing',\
'claimant asserted the cause of action of breach of contract',\
'breach of implied contract',\
'breach of written contract',\
'tortious interference',\
'tortious interference with contract',\
'breach of duty of good faith fair dealing'],\
'Claim Type 2: QuasiContractual Claims':['promissory estoppel',\
'quantum meruit',\
'money had received'],
'Claim Type 3: Breach of Fiduciary Duty':['breach of fiduciary duty',\
'breach of fiduciary duties'],\
'Claim Type 4: Negligence':['negligence',\
'gross negligence',\
'negligence gross negligence'],\
'Claim Type 5: Failure to Supervise/Negligent Supervision':['failure to supervise',\
'negligent supervision',\
'respondeat superior',\
'control person liability',\
'professional negligence',\
'vicarious liability',\
'failure to supen ise',\
'negligent hiring',\
'failure to supervise control',\
'negligent supen ision'],\
'Claim Type 6: Fraud':['fraud',\
'common law fraud',\
'fraudulent inducement',\
'constructive fraud',\
'fraud in the inducement',\
'securities fraud',\
'fraud deceit',\
'deceit',\
'manipulation'],\
'Claim Type 7: Unjust Enrichment':['unjust enrichment'],\
'Claim Type 8: Suitability or Unsuitability':['unsuitability',\
'suitability',\
'unsuitable investments',\
'unsuitabiiity'],\
'Claim Type 9: Misrepresentation':['negligent misrepresentation',\
'misrepresentation',\
'misrepresentations',\
'misrepresentations omissions',\
'fraudulent misrepresentation',\
'omissions',\
'omission of facts'],\
'Claim Type 10: Unauthorized Trading':['unauthorized trading',\
'unauthorized transactions'],\
'Claim Type 11: Churning':['churning',\
'excessive trading'],\
'Claim Type 12: Failure to Execute':['failure to execute'],\
'Claim Type 13: Breach of Promissory Note':['breach of promissory note',\
'breach of promissory notes',\
'note',\
'claimant asserted the cause of action of breach of promissory note',\
'the note',\
'breach of promissory note unjust enrichment',\
'note two',\
'note one',\
'breach of promissory note dated november'],\
'Claim Type 14: Conversion 1':['conversion'],\
'Claim Type 15: Violation of Securities Laws/Regulations':['violation of finra rules',\
'violation of finra rule',\
'violations of federal securities laws',\
'violation of state federal securities laws',\
'violation of the florida securities investor protection act',\
'violation of federal securities laws'],\
'Claim Type 16: Employment Related Claims':['wrongful termination',\
'breach of employment contract',\
'misappropriation of trade secrets',\
'constructive discharge',\
'violation of new york labor law',\
'breach of duty of loyalty',\
'tortious interference with business relationships',\
'breach of employment agreement'],\
'Claim Type 17: Other':['defamation',\
'unfair competition',\
'indemnification',\
'intentional infliction of emotional distress',\
'elder abuse']}

In [ ]:
def categorize_claims(row):
    claims_present =[]
    all_claims = row['Claim_Type_cleaned']
    for cl in all_claims:
        for k,v in claim_dict.items():
            for val in v:
                if Levenshtein.ratio(cl,val) >= 0.95:
                    claims_present.append(k)
    return claims_present

m_df['claim_categories'] = m_df.apply(categorize_claims,axis=1)        

In [ ]:
for col in claim_dict.keys():
    m_df[col] = None

for i in m_df.index:
    if (i%1000==0):
        print (i)
    all_claims = m_df.loc[i,'claim_categories']
    for col in claim_dict.keys():
        if col in all_claims:
            m_df.loc[i,col] = 1
        else:
            m_df.loc[i,col] = 0

### Cleaning and Getting Outcomes

In [ ]:
df1 = m_df[['AwardID','new_group','Region','Claim Type 1: Breach of Contract,Breach of Implied Contract',\
       'Claim Type 2: QuasiContractual Claims',\
       'Claim Type 3: Breach of Fiduciary Duty', 'Claim Type 4: Negligence',\
       'Claim Type 5: Failure to Supervise/Negligent Supervision',\
       'Claim Type 6: Fraud', 'Claim Type 7: Unjust Enrichment',\
       'Claim Type 8: Suitability or Unsuitability',\
       'Claim Type 9: Misrepresentation',\
       'Claim Type 10: Unauthorized Trading', 'Claim Type 11: Churning',\
       'Claim Type 12: Failure to Execute',\
       'Claim Type 13: Breach of Promissory Note',\
       'Claim Type 14: Conversion 1',\
       'Claim Type 15: Violation of Securities Laws/Regulations',\
       'Claim Type 16: Employment Related Claims', 'Claim Type 17: Other','loser']]
df1 = df1.drop_duplicates(keep=False)
df2 = df2[['AwardID','loc','AWARD-claims denied','expungement','claimant type']]
df2 = df2.drop_duplicates(keep=False)
df = pd.merge(df1,df2, on ='AwardID').reset_index()

In [ ]:
df['winner'] = 'Respondent'
for i in df.index:
    df_loser = df.loc[i,'loser']
    if df_loser=='Respondent':
        df.loc[i,'winner'] = 'Claimant'
    elif (df.loc[i,'AWARD-claims denied']==0.0):
        df.loc[i,'winner'] = 'Claimant'
df.to_csv('data/master_nov_26_20_41.csv')       
    